In [1]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

# -*- coding: utf-8 -*-
# @Author   : Yi Li (liyi_best@foxmail.com)
# @Time     : 2022/8/24 8:49
# @File     : 快速入门.ipynb
# @Project  : ai_quant_trade
# Copyright (c) Personal 2022 Yi Li
# Function Description: Fast tutorial for qlib
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#  本样例基于qlib/examples/workflow_by_code.py，进行了小修改和注释加入
#  本样例仅是一个快速入门样例，详细使用查看后续章节

# 1. 数据下载
* 通过qlib库获取数据：from qlib.tests.data import GetData
* 数据从微软官网获取：REMOTE_URL = "http://fintech.msra.cn/stock_data/downloads"
* 下载后默认存放在"~/.qlib/qlib_data/cn_data"
* 压缩包命名格式：20220701081835_qlib_data_simple_cn_1d_latest
* 之后自动解压

In [2]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

# use default data
# NOTE: 也可以通过qlib源码运行获取数据: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    # sys.path.append(str(scripts_dir))
    from qlib.tests.data import GetData
    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

[26088:MainThread](2022-09-13 22:30:44,307) INFO - qlib.Initialization - [config.py:402] - default_conf: client.
[26088:MainThread](2022-09-13 22:30:44,310) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[26088:MainThread](2022-09-13 22:30:44,311) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/pc/.qlib/qlib_data/cn_data')}


# 2. 模型训练
注意：不太友好的一点，数据加载过程没有进度条，如果数据多，加载过程可能稍长
* 对基本数据，计算技术指标，获取alpha158因子
* 使用LightGBM，进行回归预测

In [3]:
# market未选股的股票池，benchmark为参考的指数基准
market = "csi300"  # 沪深300股票池代码，在instruments文件夹下有对应的csi300.txt
benchmark = "SH000300"   # 沪深300指数

# 数据配置信息
#  start/end是总数据时间段，包含训练和测试
#  fit是train/valid的时间
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

# 模型及数据配置信息
# qlib.contrib.model.gbdt继承了qlib/model/base.py
task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}

# 数据和模型初始化
# 使用上面json中定义的模块路径和名称初始化类
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model, 通过R控制流程
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id  # 获取id

# 3. 预测, 回测 & 分析
如下json中：
* backtest：设置回测的时间段、账户信息和各种费用等回测条件
* executor：选择了模拟交易，按天执行，根据策略信号进行执行
* strategy: 给出决策，TopkDropoutStrategy，即基于模型对所有股票预测的得分，
  买入前k支股票，卖掉持仓的N支股票

计算时，会导入交易日的txt文件（详见数据描述的文档），根据测试日期获取
索引位置，之后按索引位置顺序执行模拟交易，用交易日减最小数据单位（即1日），
模型用前一天的值进行预测

In [5]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()


[12372:MainThread](2022-09-02 08:07:10,533) INFO - qlib.workflow - [expm.py:315] - <mlflow.tracking.client.MlflowClient object at 0x00000230AFF34EE0>
[12372:MainThread](2022-09-02 08:07:10,554) WARNING - qlib.workflow - [expm.py:195] - No valid experiment found. Create a new experiment with name backtest_analysis.
[12372:MainThread](2022-09-02 08:07:10,568) INFO - qlib.workflow - [exp.py:257] - Experiment 2 starts running ...
[12372:MainThread](2022-09-02 08:07:10,637) INFO - qlib.workflow - [recorder.py:293] - Recorder 4631afb3c47a4277b5e937a89fc9e9e4 starts running under Experiment 2 ...
[12372:MainThread](2022-09-02 08:07:12,517) INFO - qlib.workflow - [record_temp.py:194] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 2


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2017-01-03 SH600000   -0.036180
           SH600008    0.012532
           SH600009    0.036990
           SH600010   -0.020218
           SH600015   -0.129913


[12372:MainThread](2022-09-02 08:07:12,678) INFO - qlib.backtest caller - [__init__.py:83] - Create new exchange
[12372:MainThread](2022-09-02 08:13:11,120) WARNING - qlib.BaseExecutor - [executor.py:111] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x00000230AFF453D0>


backtest loop:   0%|          | 0/871 [00:00<?, ?it/s]

D:\Users\pc\anaconda3\lib\site-packages\qlib\utils\index_data.py:480: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)


KeyboardInterrupt: 

# analyze graphs

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ['label']

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)